# HDT4 PREDICCIÓN

In [4]:
import nltk
from nltk import lm
from nltk.corpus import stopwords
from nltk.book import FreqDist
from nltk.util import ngrams
import matplotlib.pyplot as plt

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
tweets = []
with open('./clean_data/en_US.twitter.txt', encoding="utf-8") as f:
    tweets = f.read()
len(tweets)

5584472

### N Gramas

In [18]:
tweets_1_gram = list(ngrams(tweets, 1))
tweets_2_gram = list(ngrams(tweets, 2))
tweets_3_gram = list(ngrams(tweets, 3))

In [19]:
tweets_1_gram[:10]

[('a',),
 ('w',),
 ('w',),
 (' ',),
 ('n',),
 ('o',),
 ('o',),
 (' ',),
 ('h',),
 ('e',)]

In [9]:
from nltk import word_tokenize, sent_tokenize
tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(tweets)]

In [10]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(3, tokenized_text)

In [11]:
from nltk.lm import MLE
mle = MLE(3)

In [12]:
mle.fit(train, vocab)
print(mle.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 68850 items>


In [13]:
mle.score('you', 'how are'.split())

0

In [17]:
mle.generate(3, 'how are'.split())

['actually', 'wrote', 'article']